In [ ]:
!pip install spacy
!pip install -qU pypdf2
!pip install langchain
!pip install langchain_community
!pip install pypdf

In [ ]:
from google.colab import drive
from langchain_community.document_loaders import PyPDFLoader
from spacy.tokens import DocBin
import json
import numpy as np
import os
import pandas as pd
import random
import re
import spacy
import spacy.util
import torch
drive.mount("/content/drive/") # load dataset drive

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
base_path = "/content/drive/MyDrive/Tubes_NLP"
dataset_path = "/content/drive/MyDrive/Tubes_NLP/Datasets"
test_dataset_path = "/content/drive/MyDrive/Tubes_NLP/Test Datasets"

In [ ]:

import spacy
from spacy.tokens import DocBin

json_files = [f"{base_path}/json/acetone.json",
              f"{base_path}/json/ammonium-chloride.json",
              f"{base_path}/json/benzyl-alcohol.json",
              f"{base_path}/json/caffeine.json",
              f"{base_path}/json/dichloromethane.json",
              f"{base_path}/json/ethanol.json",
              f"{base_path}/json/heptane.json",
              f"{base_path}/json/hexane.json",
              f"{base_path}/json/mct-oil-regular.json",
              f"{base_path}/json/methanol.json",
              f"{base_path}/json/sodium-chloride.json",
              f"{base_path}/json/tetrahydrofuran.json",
              f"{base_path}/json/triethanolamine.json",
              f"{base_path}/json/water.json",
              f"{base_path}/json/xylenes.json",]

doc_bin = DocBin()
nlp = spacy.blank("en") l

for file in json_files:
    with open(file, "r") as f:
        data = json.load(f)
        annotations = data["annotations"]
        for annotation in annotations:
            text = annotation[0]
            entities = annotation[1]
            doc = nlp.make_doc(text)
            ents = []
            for start, end, label in entities["entities"]:
                span = doc.char_span(start, end, label=label)
                if span:
                    ents.append(span)
            doc.ents = ents
            doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./train.spacy --output ./output --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    854.78    0.00    0.00    0.00    0.00
 13     200       4085.41  33117.26   96.40   94.15   98.77    0.96
 26     400       5998.59   1411.07   96.76   93.96   99.73    0.97
 40     600       1142.96   1194.54   96.70   93.61  100.00    0.97
 53     800         98.20    917.24   95.71   98.70   92.91    0.96
 66    1000        103.45    897.30   97.52   95.91   99.18    0.98
 80    1200         82.31    807.01   97.46   95.66   99.32    0.97
 93    1400         94.76    824.70   97.52   96.03   99.05    0.98
106    1600         87.71   

In [ ]:
import PyPDF2

pdf_path = f"{test_dataset_path}/getpdf.pdf"

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

pdf_text = extract_text_from_pdf(pdf_path)

In [ ]:
import spacy

nlp = spacy.load("./output/model-last")
doc = nlp(pdf_text)

for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")

Entity: Calcium Hydroxide, Label: CHEMICAL_NAME
Entity: 1305-62-0, Label: CAS_NUMBER
Entity: soap, Label: CHEMICAL_NAME
Entity: water., Label: CHEMICAL_NAME
Entity: 1305-62-0, Label: CAS_NUMBER
Entity: water., Label: CHEMICAL_NAME
Entity: water, Label: CHEMICAL_NAME
Entity: Calcium, Label: CHEMICAL_NAME
Entity: water, Label: CHEMICAL_NAME
Entity: 1305-62-0, Label: CAS_NUMBER
Entity: 109  
n, Label: CAS_NUMBER
Entity: Water, Label: CHEMICAL_NAME
Entity: n-octanol/water, Label: CHEMICAL_NAME


In [ ]:
import json
import spacy
from spacy.tokens import DocBin

json_files = [f"{base_path}/json/testpdf (1).json",
              f"{base_path}/json/testpdf (2).json",
              f"{base_path}/json/testpdf (3).json",
              f"{base_path}/json/testpdf (4).json",
              f"{base_path}/json/testpdf (5).json",
              f"{base_path}/json/testpdf (6).json",
              f"{base_path}/json/testpdf (7).json",
              f"{base_path}/json/testpdf (8).json",
              f"{base_path}/json/testpdf (9).json",
              f"{base_path}/json/testpdf (10).json"]

doc_bin = DocBin()
nlp = spacy.blank("en")

for file in json_files:
    with open(file, "r") as f:
        data = json.load(f)
        annotations = data["annotations"]
        for annotation in annotations:
            text = annotation[0]
            entities = annotation[1]
            doc = nlp.make_doc(text)
            ents = []
            for start, end, label in entities["entities"]:
                span = doc.char_span(start, end, label=label)
                if span:
                    ents.append(span)
            doc.ents = ents
            doc_bin.add(doc)

doc_bin.to_disk("test.spacy")

In [ ]:
!python -m spacy evaluate ./output/model-last ./test.spacy --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   80.11 
NER R   43.41 
NER F   56.31 
SPEED   15459 


=============================== NER (per type) ===============================

                     P       R       F
CHEMICAL_NAME    84.25   41.80   55.87
CAS_NUMBER       69.39   85.00   76.40
COMPANY           0.00    0.00    0.00
FORMULA         100.00   36.36   53.33

